In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%cd drive/MyDrive/data/aT/scaled_data

/content/drive/MyDrive/data/aT/scaled_data


In [ ]:
%ls

answer_example.csv            model_weights_28.bias_3.pth  set_4/
aT_test_raw/                  model_weights_28.bias_4.pth  set_5/
Linear_weight/                model_weights_28.bias_5.pth  set_6/
model_weights_28.bias_0.pth   model_weights_28.bias_6.pth  set_7/
model_weights_28.bias_10.pth  model_weights_28.bias_7.pth  set_8/
model_weights_28.bias_11.pth  model_weights_28.bias_8.pth  set_9/
model_weights_28.bias_12.pth  model_weights_28.bias_9.pth  test/
model_weights_28.bias_13.pth  set_0/                       train/
model_weights_28.bias_14.pth  set_1/                       wandb/
model_weights_28.bias_1.pth   set_2/
model_weights_28.bias_2.pth   set_3/


In [ ]:
!wandb login

wandb: Currently logged in as: deep-overflow (deepintodeep). Use `wandb login --relogin` to force relogin


In [ ]:
import pandas as pd
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split
import random
import os
import wandb
from tqdm import tqdm

import torch
import torch.nn as nn # Neural Network
import torch.optim as optim # Optimizer
import torch.optim.lr_scheduler as lr_scheduler # Scheduler
from torch.utils.data import Dataset, DataLoader # Data

In [ ]:
# 학습에 사용할 데이터 파일 (csv)
data_list = glob('.//train/*.csv')

tr_del_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 '] # train 에서 사용하지 않는 열
ts_del_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 ', '해당일자_전체평균가격(원)'] # test 에서 사용하지 않는 열
check_col = ['일자구분_중순', '일자구분_초순', '일자구분_하순','월구분_10월', '월구분_11월', '월구분_12월', '월구분_1월', '월구분_2월', '월구분_3월', 
             '월구분_4월','월구분_5월', '월구분_6월', '월구분_7월', '월구분_8월', '월구분_9월'] # 열 개수 맞추기

def time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in range(len(df) - seqence_length):
        tmp = df[index: index + seqence_length].values
        tmp = np.vstack(tmp).astype(np.float32)
        result.append(tmp)

    return np.array(result)

In [ ]:
def right_value(a, i):
    for j in range(i + 1, len(a)):
        if a[j] != 0:
            return a[j]
    return None

def left_value(a ,i):
    for j in range(0, i):
        if a[i - j - 1] != 0:
            return a[i - j - 1]
    return None

def nearest_value(a, i):
    size = len(a)
    rvalue = right_value(a, i)
    lvalue = left_value(a, i)

    if rvalue == None:
        if lvalue == None:
            return None
        else:
            return lvalue
    else:
        if lvalue == None:
            return rvalue
        else:
            return rvalue

In [ ]:
import torch

class Data:
    def __init__(self, data_path='./train/', item_idx=0, split_ratio=0.2):
        # 파일 경로 ex) item_idx = 0인 경우, file_path='./train/train_0.csv'이다.
        file_path = os.path.join(data_path, f'train_{item_idx}.csv')

        # csv 파일 읽기
        data_pd = pd.read_csv(file_path)

        # 데이터 전처리
        for column in data_pd.columns:
            data_pd[column] = data_pd[column].replace({' ': np.nan})
        
        data_pd.drop(tr_del_list, axis=1, inplace=True)
        data_pd.set_index('datadate', drop=True, inplace=True)

        data_pd = data_pd.fillna(0)

        x_pd, y_pd = data_pd[[i for i in data_pd.columns if i != '해당일자_전체평균가격(원)']], data_pd['해당일자_전체평균가격(원)']

        y_t = np.array(y_pd)

        y_pd = y_pd[14:]

        x_np = time_window(x_pd, 13, 1)
        y_np = time_window(y_pd, 27, 1)

        x_np = x_np[:len(y_np)]
        y_np = y_np.reshape(-1, 28)
        print(len(x_np))

        # 변동률 예측
        print(y_np[0])
        print(y_t[0])
        for i in range(len(y_np)):
            for j in range(28):
                if y_t[i] == 0:
                    value_t = nearest_value(y_t, i)
                else:
                    value_t = y_t[i]

                y_np[i][j] = y_np[i][j] - value_t
                y_np[i][j] = y_np[i][j] / value_t

        print(y_np[0])
        self.data = {
            'inputs': x_np,
            'labels': y_np,
        }

        self.split_ratio = split_ratio

        if self.split_ratio > 0:
            self.train_val_split()
    
    def train_val_split(self):
        x_train, x_val, y_train, y_val = train_test_split(
            self.data['inputs'],
            self.data['labels'],
            test_size=self.split_ratio,
            random_state=42
        )

        self.train_data = (x_train, y_train)
        self.val_data = (x_val, y_val)

    def get_train(self, use_tensor=False):
        if use_tensor:
            return torch.tensor(self.train_data[0], dtype=torch.float32), torch.tensor(self.train_data[1], dtype=torch.float32)
        else:
            return self.train_data
    
    def get_val(self, use_tensor=False):
        if use_tensor:
            return torch.tensor(self.val_data[0], dtype=torch.float32), torch.tensor(self.val_data[1], dtype=torch.float32)
        else:
            return self.val_data


In [ ]:
class TestData:
    pass

In [ ]:
data = Data()

1419
[   0.     2916.061  2313.3643 3045.0603 3214.291  4387.5825 1631.7
    0.     2700.754  1101.2709    0.        0.        0.        0.
    0.        0.        0.     5475.074  2363.925  3093.0178 2249.058
    0.     2783.1553 1031.9697 3062.8455 2593.413  2983.8428 2347.6807]
0.0
[-0.99999994 -0.8592107  -0.88830936 -0.8529825  -0.844812   -0.78816473
 -0.9212205  -0.99999994 -0.8696059  -0.94683    -0.99999994 -0.99999994
 -0.99999994 -0.99999994 -0.99999994 -0.99999994 -0.99999994 -0.73566
 -0.88586825 -0.8506671  -0.8914141  -0.99999994 -0.8656275  -0.9501759
 -0.85212386 -0.8747884  -0.8559382  -0.8866525 ]


In [ ]:
class TorchDataset(Dataset):
    def __init__(self, x_tensor, y_tensor):
        super().__init__()
        self.x = x_tensor
        self.y = y_tensor
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
    
    def __len__(self):
        return len(self.x)

In [ ]:
class Flag:
    def __init__(self, flags):
        for key, value in flags.items():
            if isinstance(value, dict):
                self.__dict__[key] = Flag(value)
            else:
                self.__dict__[key] = value

In [ ]:
def train_and_val(flags):
    name = flags['name']

    wandb.init(
        project="Nong_Linear",
        entity='deep-overflow',
        config=flags,
        name=name
    )

    flags = Flag(flags)

    for item_idx in range(37):
        data = Data(item_idx=item_idx)

        # Data
        train_data = data.get_train(use_tensor=True)
        val_data = data.get_val(use_tensor=True)
        
        # Dataset
        train_dataset = TorchDataset(train_data[0], train_data[1])
        val_dataset = TorchDataset(val_data[0], val_data[1])

        # Dataloader
        train_dataloader = DataLoader(
            dataset=train_dataset,
            batch_size=flags.batch_size,
            shuffle=True
        )
        val_dataloader = DataLoader(
            dataset=val_dataset,
            batch_size=flags.batch_size,
            shuffle=False
        )

        for inputs, labels in train_dataloader:
            print(inputs.shape)
            print(labels.shape)
            break

        net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(14 * 49, flags.model.hid_dim),
            nn.BatchNorm1d(flags.model.hid_dim),
            nn.ReLU(),
            nn.Dropout(flags.dropout),
        )

        for _ in range(flags.model.nlayers - 2):
            net.append(nn.Linear(flags.model.hid_dim, flags.model.hid_dim))
            # net.append(nn.BatchNorm1d(flags.model.hid_dim))
            net.append(nn.ReLU())
            net.append(nn.Dropout(flags.dropout))
        
        net.append(nn.Linear(flags.model.hid_dim, 28))

        for name, param in net.named_parameters():
            if name.split('.')[-1] == 'bias':
                continue
            nn.init.xavier_uniform_(param)

        criterion = nn.L1Loss()

        optimizer = optim.Adam(
            params=net.parameters(), 
            lr=flags.lr,
            betas=(0.9, 0.999),
        )

        def lr_schedule_fn(epoch):
            if epoch < 15:
                return 1.5 # 1e-4 * 1.5
            elif epoch >= 15 and epoch < 250:
                return 1.0
            elif epoch >= 250 and epoch < 350:
                return 1.5
            else:
                return 1.0

        scheduler = lr_scheduler.LambdaLR(
            optimizer=optimizer,
            lr_lambda=lr_schedule_fn
        )

        device = 'cuda' if torch.cuda.is_available() else 'cpu'

        net = net.to(device)

        best_val_loss = 987654321.0

        for epoch in range(flags.epochs):
            epoch_train_loss = 0.0

            for inputs, labels in train_dataloader:
                inputs, labels = inputs.to(device), labels.to(device)

                with torch.set_grad_enabled(True):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                epoch_train_loss += loss.item()
            
            scheduler.step()

            if (epoch + 1) % 10 == 0:
                print(f'[epoch : {epoch + 1} / {flags.epochs}] Train Loss : {epoch_train_loss / len(train_dataloader)}')

            epoch_val_loss = 0.0

            for inputs, labels in val_dataloader:
                inputs, labels = inputs.to(device), labels.to(device)

                with torch.no_grad():
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)
                
                epoch_val_loss += loss.item()
            
            wandb.log({
                'train_loss': epoch_train_loss / len(train_dataloader),
                'train_lr': optimizer.param_groups[0]['lr'],
                'val_loss': epoch_val_loss / len(val_dataloader),
            })

            val_loss = epoch_val_loss / len(val_dataloader)

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                print(f'Save item: {item_idx} / best_val_loss: {best_val_loss}')
                torch.save(net.state_dict(), f'./model_weights_{name}_{item_idx}.pth')

flags = {
    'name': 'Exp_val_y_variance_001',
    'epochs': 500,
    'lr': 1e-4,
    'batch_size': 64,
    'data_num': 0,
    'model': {
        'activation': 'relu',
        'nlayers': 10,
        'use_BatchNorm': True,
        'hid_dim': 1024,
        'weight_init': 'xavier_uniform'
    },
    'optim': 'Adam',
    'criterion': 'L1Loss',
    'dropout': 0.5,
    'lr_scheduler': {
        'method': 'Lambda'
    }
}

train_and_val(flags)

train_loss,████████▃▂▂▂▂▁▂▂▁▁▁▁▁▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train_lr,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁████████▁▁▁▁▁▁▁▁▁▁▁
val_loss,████████▃▂▂▁▃▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▁▂▁
train_loss,135.09823
train_lr,0.0001
val_loss,187.83022


1419
[   0.     2916.061  2313.3643 3045.0603 3214.291  4387.5825 1631.7
    0.     2700.754  1101.2709    0.        0.        0.        0.
    0.        0.        0.     5475.074  2363.925  3093.0178 2249.058
    0.     2783.1553 1031.9697 3062.8455 2593.413  2983.8428 2347.6807]
0.0
[-0.99999994 -0.8592107  -0.88830936 -0.8529825  -0.844812   -0.78816473
 -0.9212205  -0.99999994 -0.8696059  -0.94683    -0.99999994 -0.99999994
 -0.99999994 -0.99999994 -0.99999994 -0.99999994 -0.99999994 -0.73566
 -0.88586825 -0.8506671  -0.8914141  -0.99999994 -0.8656275  -0.9501759
 -0.85212386 -0.8747884  -0.8559382  -0.8866525 ]
torch.Size([64, 14, 49])
torch.Size([64, 28])
Save item: 0 / best_val_loss: 0.4793683111667633
Save item: 0 / best_val_loss: 0.47562294602394106
Save item: 0 / best_val_loss: 0.4753348231315613
Save item: 0 / best_val_loss: 0.4729121387004852
Save item: 0 / best_val_loss: 0.4726067900657654
Save item: 0 / best_val_loss: 0.4712198436260223
Save item: 0 / best_val_loss: 0.470

In [ ]:
def make_Tensor(array):
    return torch.from_numpy(array)


def astype_data(data):
    df = data.astype(np.float32)
    return make_Tensor(df)


class testDataset(Dataset):
    def __init__(self, data):
        zero_csv = [0 for i in range(14)]
        df = pd.read_csv(data)

        if len(df) == 0:
            print('no data in Dataset!!')
            print(df)
            df['zero_non'] = zero_csv
            print(df)
            df = df.fillna(0)
            print(df)
            df.drop('zero_non', axis=1, inplace=True)
            df.drop('Unnamed: 0', axis=1, inplace=True)
            print(df)

        file_number = data.split('test_')[1].split('.')[0]

        # 사용할 열 선택, index 설정
        df.drop(ts_del_list, axis=1, inplace=True)
        df.set_index('datadate', drop=True, inplace=True)

        # train input 과 형상 맞추기
        add_col = [i for i in check_col if i not in df.columns]

        for a in add_col:
            df[a] = 0

        # ' ' -> nan 으로 변경
        for a in df.columns:
            df[a] = df[a].replace({' ': np.nan})

        # nan 처리
        df = df.fillna(0)

        # x_test  생성
        self.df_test = astype_data(df.values.reshape(1, df.values.shape[0], df.values.shape[1]))

    def __len__(self):
        return len(self.df_test)

    def __getitem__(self, idx):
        return self.df_test[idx]

In [ ]:
def submit(flags):
    flags = Flag(flags)

    result_np = np.zeros((1, 28), dtype=np.float32)
    for item_idx in range(37):

        net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(14 * 49, flags.model.hid_dim),
            # nn.BatchNorm1d(flags.model.hid_dim),
            nn.ReLU(),
            nn.Dropout(flags.dropout),
        )

        for _ in range(flags.model.nlayers - 2):
            net.append(nn.Linear(flags.model.hid_dim, flags.model.hid_dim))
            # net.append(nn.BatchNorm1d(flags.model.hid_dim))
            net.append(nn.ReLU())
            net.append(nn.Dropout(flags.dropout))
        
        net.append(nn.Linear(flags.model.hid_dim, 28))

        net.load_state_dict(torch.load(f'./model_weights_28.bias_{item_idx}.pth'))

        net = net.to('cuda')

        for set_num in range(10):
            file_path = f'./test/set_{set_num}/test_{item_idx}.csv'
            test_dataset = testDataset(file_path)

            inputs = test_dataset[0].reshape(1, 14, 49).to('cuda')

            with torch.no_grad():
                outputs = net(inputs)            
            output_np = outputs.cpu().detach().numpy()
            save_df = pd.DataFrame(output_np).T
            save_df.to_csv(f'./set_{set_num}/predict_{item_idx}.csv', index=False)
            print(f'Save Result set: {set_num}, item: {item_idx}')

flags = {
    'name': 'Exp_val_y_variance_001',
    'epochs': 500,
    'lr': 1e-4,
    'batch_size': 64,
    'data_num': 0,
    'model': {
        'activation': 'relu',
        'nlayers': 10,
        'hid_dim': 1024,
        'weight_init': 'xavier_uniform'
    },
    'optim': 'Adam',
    'criterion': 'L1Loss',
    'dropout': 0.5,
    'lr_scheduler': {
        'method': 'Lambda'
    }
}
submit(flags)

Save Result set: 0, item: 0
Save Result set: 1, item: 0
Save Result set: 2, item: 0
Save Result set: 3, item: 0
Save Result set: 4, item: 0
Save Result set: 5, item: 0
Save Result set: 6, item: 0
Save Result set: 7, item: 0
Save Result set: 8, item: 0
Save Result set: 9, item: 0
Save Result set: 0, item: 1
Save Result set: 1, item: 1
Save Result set: 2, item: 1
Save Result set: 3, item: 1
Save Result set: 4, item: 1
Save Result set: 5, item: 1
Save Result set: 6, item: 1
Save Result set: 7, item: 1
Save Result set: 8, item: 1
Save Result set: 9, item: 1
Save Result set: 0, item: 2
Save Result set: 1, item: 2
Save Result set: 2, item: 2
Save Result set: 3, item: 2
Save Result set: 4, item: 2
Save Result set: 5, item: 2
Save Result set: 6, item: 2
Save Result set: 7, item: 2
Save Result set: 8, item: 2
Save Result set: 9, item: 2
Save Result set: 0, item: 3
Save Result set: 1, item: 3
Save Result set: 2, item: 3
Save Result set: 3, item: 3
Save Result set: 4, item: 3
Save Result set: 5, 

In [ ]:
for k in tqdm(range(10)):

  globals()[f'set_df_{k}'] = pd.DataFrame()
  answer_df_list = glob(f'./set_{k}/*.csv') # 예측한 결과 불러오기
  pum_list = glob(f'./aT_test_raw/sep_{k}/*.csv') # 기존 test input 불러오기
  pummok = [a for a in pum_list if 'pummok' in a.split('/')[-1]]

  for i in answer_df_list:
    df = pd.read_csv(i)
    number = i.split('_')[-1].split('.')[0]

    base_number = 0
    for p in pummok:
      if number == p.split('_')[-1].split('.')[0]:
        pum_df = pd.read_csv(p)

        if len(pum_df) != 0:
           base_number = pum_df.iloc[len(pum_df)-1]['해당일자_전체평균가격(원)']  # 기존 각 sep 마다 test input의 마지막 target 값 가져오기 (변동률 계산을 위해)
        else:
          base_number = np.nan

    globals()[f'set_df_{k}'][f'품목{number}']  = [base_number] + list(df[df.columns[-1]].values) # 각 품목당 순서를 t, t+1 ... t+28 로 변경

  globals()[f'set_df_{k}'] = globals()[f'set_df_{k}'][[f'품목{col}' for col in range(37)]] # 열 순서를 품목0 ~ 품목36 으로 변경

100%|██████████| 10/10 [00:39<00:00,  3.98s/it]


In [ ]:
set_df_0

,품목0,품목1,품목2,품목3,품목4,품목5,품목6,품목7,품목8,품목9,...,품목27,품목28,품목29,품목30,품목31,품목32,품목33,품목34,품목35,품목36
0,3871.125000,1362.117613,2909.783785,3400.075583,3947.809169,9253.947514,2717.280000,3361.030923,4911.899864,1173.018633,...,8640.811309,602.005658,1105.412623,1566.274239,3633.464557,5454.710444,5619.188362,5230.620027,2905.100888,2087.675036
1,-0.098704,-0.208349,-0.030669,-0.179442,-0.033723,-0.088001,-0.095634,-0.036049,-0.127025,0.014201,...,-0.171518,-0.092869,-0.123811,0.013422,-0.101043,0.027667,-0.092294,-0.057289,-0.089267,-0.080421
2,-0.105068,-0.194260,-0.045841,-0.185040,-0.024620,-0.097661,-0.145952,-0.021373,-0.128772,0.011254,...,-0.171184,-0.102812,-0.146323,0.021701,-0.119364,0.030727,-0.093349,-0.081633,-0.100971,-0.091578
3,-0.104312,-0.213983,-0.040041,-0.202736,-0.032060,-0.093784,-0.110657,-0.045481,-0.128919,0.005640,...,-0.187573,-0.111513,-0.176994,0.017166,-0.127064,0.037848,-0.082695,-0.086575,-0.106187,-0.105724
4,-0.096159,-0.221220,-0.031434,-0.197726,-0.045425,-0.103915,-0.054150,-0.030704,-0.149009,0.010941,...,-0.178969,-0.089716,-0.153893,0.019843,-0.137207,0.023684,-0.092194,-0.089181,-0.108863,-0.083952
5,-0.079600,-0.219619,-0.034849,-0.211215,-0.031644,-0.106284,-0.107980,-0.039167,-0.129154,0.011154,...,-0.191400,-0.096434,-0.155161,0.022190,-0.121410,0.034709,-0.117058,-0.076427,-0.100425,-0.104697
6,-0.102885,-0.220557,-0.037263,-0.220034,-0.045295,-0.104895,-0.085439,-0.030747,-0.151296,0.013863,...,-0.185042,-0.085266,-0.170424,0.032275,-0.111476,0.044160,-0.121155,-0.061194,-0.102796,-0.076243
7,-0.102408,-0.215402,-0.043569,-0.236576,-0.060382,-0.107047,-0.120601,-0.015257,-0.131517,0.008270,...,-0.201968,-0.097505,-0.165630,0.032205,-0.111614,0.028726,-0.106227,-0.089874,-0.095072,-0.088554
8,-0.103783,-0.199212,-0.036895,-0.228485,-0.043003,-0.102455,-0.088212,-0.020756,-0.132866,0.017791,...,-0.190577,-0.108066,-0.159750,0.037268,-0.116338,0.025974,-0.097460,-0.099172,-0.094013,-0.086691
9,-0.111943,-0.220475,-0.048597,-0.236415,-0.050359,-0.117733,-0.129261,-0.023630,-0.135039,0.016688,...,-0.196949,-0.112684,-0.162672,0.032753,-0.122265,0.044189,-0.107037,-0.100952,-0.092463,-0.080775


In [ ]:
date = [f'd+{i}' for i in range(1,15)] + ['d+22 ~ 28 평균']


for k in range(10):
  globals()[f'answer_df_{k}'] = pd.DataFrame()
  for c in globals()[f'set_df_{k}'].columns:
    base_d = globals()[f'set_df_{k}'][c][0] # 변동률 기준 t 값

    ans_1_14 = []
    for i in range(14):
      ans_1_14.append((globals()[f'set_df_{k}'][c].iloc[i+1]- base_d)/base_d)  # t+1 ~ t+14 까지는 (t+n - t)/t 로 계산

    ans_22_28 = (globals()[f'set_df_{k}'][c][22:29].mean() - base_d)/base_d # t+22 ~ t+28은 np.mean(t+22 ~ t+28) - t / t

    globals()[f'answer_df_{k}'][f'{c} 변동률'] = ans_1_14 + [ans_22_28]
  
  globals()[f'answer_df_{k}']['Set'] = k # set 번호 설정
  globals()[f'answer_df_{k}']['일자'] = date # 일자 설정

# 위에서 계산된 변동률 들을 합쳐주는 과정

all_df =pd.DataFrame()
for i in range(10):
  if i== 0 :
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']],axis=1)
  else:
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']])


all_df = all_df[['Set','일자'] + list(all_df.columns[:-2])]
all_df.reset_index(drop=True, inplace=True)

# set, 일자 기억하기위해 따로 저장

re_set = list(all_df['Set'])
re_date = list(all_df['일자'])


# 정답 양식 불러오기
out_ans = pd.read_csv('./answer_example.csv')

# 두 dataframe 합치기 (nan + 숫자 = nan 이용)
submit_df = all_df + out_ans

submit_df['Set'] = re_set
submit_df['일자'] = re_date


# 최종 저장
submit_df.to_csv('./submit.csv',index=False)